In [1]:
import torch
import transformers
import pandas as pd
import numpy as np
import torch
import os
import sys
import torch.nn as nn
from tqdm import tqdm
from sklearn.metrics import roc_auc_score, precision_score, recall_score, precision_recall_curve, auc

模型大小的限制，模型输入的限制，然后各种限制。
pip tokenizers datasets transformers

In [2]:
from transformers import AutoModel
from transformers import AutoTokenizer
#seyonec/PubChem10M_SMILES_BPE_450k
from transformers import RobertaModel,BertModel

In [5]:
model = RobertaModel.from_pretrained('roberta-base')

Downloading: 100%|██████████| 481/481 [00:00<00:00, 240kB/s]
Downloading: 100%|██████████| 501M/501M [17:34<00:00, 475kB/s] 
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

Downloading: 100%|██████████| 899k/899k [00:04<00:00, 201kB/s]
Downloading: 100%|██████████| 456k/456k [00:08<00:00, 53.3kB/s]
Downloading: 100%|██████████| 1.36M/1.36M [00:47<00:00, 28.7kB/s]


In [8]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

train = pd.read_csv('../train.csv', sep='\t')
test = pd.read_csv('../test.csv', sep='\t')
sub = pd.read_csv('../sample_submit.csv')

# 拼接title与abstract
train['text'] = train['title'] + ' ' + train['abstract']
test['text'] = test['title'] + ' ' + test['abstract']

label_id2cate = dict(enumerate(train.categories.unique()))
label_cate2id = {value: key for key, value in label_id2cate.items()}

train['label'] = train['categories'].map(label_cate2id)

train = train[['text', 'label']]
train_y = train["label"]
# train_df = train[['text', 'label']][:45000]
# eval_df = train[['text', 'label']][45000:]


In [1]:
# label_id2cate
# model

In [10]:
index = np.random.permutation(range(len(train)))
train_data = train.iloc[index[:int(0.9*len(train))]].reset_index(drop = True)
val_data = train.iloc[index[int(0.9*len(train)):]].reset_index(drop = True)

In [11]:
from torch.utils.data import Dataset,DataLoader,TensorDataset
class My_dataset(Dataset):
    def __init__(self,data_len):
        super(My_dataset,self).__init__()
            
        self.data = data_len
    def __getitem__(self,idx):
#         label = self.data.iloc[idx][2]
#         mol = self.data.iloc[idx][0]
#         pro = self.data.iloc[idx][1]
#         pro = ' '.join(pro)
#         mol_inputs = tokenizer_mol(mol)['input_ids']
#         mol_inputs_mask = tokenizer_mol(mol)['attention_mask']
#         pro_inputs = tokenizer_pro(pro)['input_ids']
#         pro_inputs_mask = tokenizer_pro(pro)['attention_mask']
#         pro_inputs_type = tokenizer_pro(pro)['token_type_ids']
        
        return self.data[idx]#mol,'_'.join(pro),label#torch.tensor(),torch.tensor(),torch.tensor(tss),torch.tensor(feats,dtype = torch.float32)
    def __len__(self):
        return len(self.data)
#class my_dataset(nn.)

In [25]:
val_dataloader = DataLoader(list(range(len(val_data)))[:64],batch_size = 32,num_workers = 0,shuffle = True)

In [26]:
train_dataloader = DataLoader(list(range(len(train_data)))[:64],batch_size = 32,num_workers = 0,shuffle = True)

In [27]:
class MLP(nn.Module):
    def __init__(self,dim_in,dim_hidden,dim_out):
        super(MLP,self).__init__()
        self.fc_1 = nn.Linear(dim_in,dim_hidden)
        self.fc_2 = nn.Linear(dim_hidden,dim_out)
    def forward(self,data):
        return self.fc_2(torch.relu(self.fc_1(data)))#return logits for crosss_entry

In [28]:
val_data

,text,label
0,The Optimization of a Novel Prismatic Drive ...,7
1,On Consensus under Polynomial Protocols In t...,36
2,Surface Approximation via Asymptotic Optimal G...,17
3,Cascaded Semantic and Positional Self-Attentio...,0
4,Efficient and Flexible Crowdsourcing of Specia...,22
...,...,...
4995,DWT Based Fingerprint Recognition using Non Mi...,3
4996,Bi-Dimensional Feature Alignment for Cross-Dom...,3
4997,Partial Truthfulness in Minimal Peer Predictio...,23
4998,Separable Four Points Fundamental Matrix We ...,3


In [41]:
modelmlp = MLP(768,768*2,39)
Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(modelmlp.parameters(),lr = 0.001)
epochs = 1
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
for epoch in range(1,epochs+1):
    print('strating training!!')
    train_loss = []
    val_loss = []
    #test_loss = 0
    S = []
    T = []
    S_train = []
    T_train = []
    modelmlp.to(device)
    modelmlp.train()
    model.eval()
    
    for batch in tqdm(train_dataloader):
        batch_data = train_data.iloc[batch].copy()
        #batch_data[1] = batch_data[1].apply(lambda x : re.sub(r"[UZOB]", "X", x))
        # mol_batch = list(batch_data[0].values)
        # pro_batch = list(batch_data[1].values)
        text_batch = list(batch_data['text'].values)
        label_batch = torch.tensor(list(batch_data['label'].values))
#         model_mol.eval()
#         model_pro.eval()
        with torch.no_grad():
            #size = [bs,768]
            model_out = model(**tokenizer(text_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True))['pooler_output']
            # #size = [bs,1024]
            # pro_out = model_pro(**tokenizer_pro(pro_batch,return_tensors = 'pt',padding = True,max_length = 1500,truncation = True))['pooler_output']
        # model_input = torch.cat([mol_out,pro_out],dim = 1)#bs,1792
        model_out = modelmlp(model_out.to(device))
        loss = Loss(model_out,label_batch.to(device))
        scores = torch.softmax(model_out,dim = 1).cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
        print('train_loss:, ', np.sum(train_loss)/len(train_loss))
        S_train.extend(scores)
        T_train.extend(label_batch.cpu().detach().numpy())
    AUC = roc_auc_score(T_train, S_train,multi_class='ovo')
    tpr, fpr, _ = precision_recall_curve(T_train, S_train)
    PRC = auc(fpr, tpr)
    #print(AUC, PRC)
    print('train_loss:, ',np.sum(train_loss)/len(train_loss),'AUC: ',AUC,'PRC: ',PRC) 
#val state
    modelmlp.eval()
    for batch in val_dataloader:

        batch_data = val_data.iloc[batch].copy()
        #batch_data[1] = batch_data[1].apply(lambda x : re.sub(r"[UZOB]", "X", x))
        # mol_batch = list(batch_data[0].values)
        # pro_batch = list(batch_data[1].values)
        text_batch = list(batch_data['text'].values)
        label_batch = torch.tensor(list(batch_data['label'].values))
        with torch.no_grad():
            #size = [bs,768]
            # mol_out = model_mol(**tokenizer_mol(mol_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True))['pooler_output']
            #size = [bs,1024]
            model_out = model_pro(**tokenizer_pro(pro_batch,return_tensors = 'pt',padding = True,max_length = 512,truncation = True))['pooler_output']
            # model_input = torch.cat([mol_out,pro_out],dim = 1)#bs,1792
            model_out = model(model_out.to(device))
            loss = Loss(model_out,label_batch.to(device))
            #scores = torch.softmax(model_out,dim = 1)[:,1]
            val_loss.append(loss.item())
            scores = torch.softmax(model_out,dim = 1).cpu().detach().numpy()
            S.extend(scores)
            T.extend(label_batch.cpu().detach().numpy())
            #print('val_loss: ',val_loss)
    AUC = roc_auc_score(T, S,multi_class='ovo')
    tpr, fpr, _ = precision_recall_curve(T, S)
    PRC = auc(fpr, tpr)
    print('val_loss: ',np.sum(val_loss)/len(val_loss),'AUC: ',AUC,'PRC: ',PRC) 
    torch.save(modelmlp,'./my_pretrain_model.pth')

  0%|          | 0/2 [00:00<?, ?it/s]

strating training!!


 50%|█████     | 1/2 [00:28<00:28, 28.14s/it]

train_loss:,  3.658642530441284


100%|██████████| 2/2 [00:55<00:00, 27.61s/it]

train_loss:,  3.6037017107009888


ValueError: Number of classes in y_true not equal to the number of columns in 'y_score'

In [35]:
AUC = roc_auc_score(T_train, S_train,multi_class='ovo')

AxisError: axis 1 is out of bounds for array of dimension 1

In [42]:
T_train.sort()
T_train

[0,
 0,
 0,
 0,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 5,
 5,
 5,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 9,
 9,
 9,
 9,
 10,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 15,
 15,
 15,
 15,
 16,
 19,
 19,
 22,
 25,
 28,
 28,
 28,
 28,
 28,
 28,
 29,
 29,
 29]

In [46]:
np.array(S_train).shape

(64, 39)

SyntaxError: invalid syntax (<ipython-input-44-a35077a93b18>, line 1)